<a href="https://colab.research.google.com/github/ajou-y0uC4N/y0uC4N/blob/main/dnn6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
import numpy as np
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import drive
drive.mount('/content/drive')


df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/team_prj/data/type6.csv")

#df = pd.get_dummies(df)
print(df.head())
df['type'] = df['type'].map({'Attack': 0, 'Normal': 1})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     type      time  isCan0  priority  messageID  isService  serviceNODEID  \
0  Normal  0.000000       1         5       1030          0              1   
1  Normal  0.000180       1         5       1030          0              1   
2  Normal  0.002787       1         5       1030          0              1   
3  Normal  0.002945       1         5       1030          0              1   
4  Normal  0.007147       1         5       1030          0              1   

   byteSize  data[0]  data[1]  data[2]  data[3]  data[4]  data[5]  data[6]  \
0         8      166       53        0        0        0        0        0   
1         7        0        0        0        0        0        0      124   
2         8      166       53        0        0        0        0        0   
3         7        0        0        0        0        0        0      125   
4         8 

In [24]:
y = df["type"].astype("category")
X = df.drop(columns=["type","time","isCan0","serviceNODEID"])#.iloc[:, :-1].values
print(X.dtypes)
print(y.unique())
print(y.dtypes)
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

# 모델 학습
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

priority        int64
messageID       int64
isService       int64
byteSize        int64
data[0]         int64
data[1]         int64
data[2]         int64
data[3]         int64
data[4]         int64
data[5]         int64
data[6]         int64
data[7]         int64
7byteNullCnt    int64
8byteNullCnt    int64
dtype: object
[1, 0]
Categories (2, int64): [0, 1]
category
X shape: (241321, 14)
y shape: (241321,)


In [25]:
print(model.summary())
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping])


loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
print(f'Test accuracy: {accuracy}')

model.save('my_dnn_model.h5')


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_48 (Dense)            (None, 64)                960       
                                                                 
 dense_49 (Dense)            (None, 32)                2080      
                                                                 
 dense_50 (Dense)            (None, 16)                528       
                                                                 
 dense_51 (Dense)            (None, 1)                 17        
                                                                 
Total params: 3585 (14.00 KB)
Trainable params: 3585 (14.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/100
4827/4827 [==============================] - 19s 3ms/step - loss: 0.1744 - binary_accuracy: 0.9332 - val_loss: 0.1646 - val_binary_acc

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = (model.predict(X_test) > 0.5).astype(np.float32)

# 평가 지표 계산
report = classification_report(y_test, y_pred, target_names=['Attack', 'Normal'])
print(report)
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

1509/1509 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

      Attack       0.99      0.98      0.99     16419
      Normal       0.99      0.99      0.99     31846

    accuracy                           0.99     48265
   macro avg       0.99      0.99      0.99     48265
weighted avg       0.99      0.99      0.99     48265

[[16169   250]
 [  209 31637]]
